# Generative example

This example uses the LangChain framework with Ollama to generate synthetic clinical notes from synthetic structured data.

Running this notebook requires an additional install of [Ollama](https://ollama.ai/) and the particular model used is `llama2:latest` (also named `llama2:7b-chat`) from the [Ollama model library](https://ollama.ai/library/llama2/tags).

Open a terminal and run `ollama pull llama2` to download the model.

## Import libraries

In [ ]:
import json
import os

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [2]:
path_input = "../../data/synthea.json"
path_output = "../../data/llm.json"

## Load input in batch

In [3]:
with open(path_input) as file:
    data = json.load(file)

batch = []

for i in range(5):
    batch.append({"data": json.dumps(data[i])})

## Load LLM

The model used to run inference can be easily swapped out using LangChain and Ollama!

Simply open a terminal and run `ollama pull <model_name:tag>` to retrieve any model from the Ollama model library and pass `<model_name:tag>` as the new argument when instantiating the LLM.

For example, if you wanted to use Mistral instead of Llama2, you would need to run `ollama pull mistral` in a terminal and set `model="mistral"` below.

In [4]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = Ollama(model="llama2", callback_manager=callback_manager)

## Construct prompt

In [5]:
template = """[INST]
<<SYS>>
You are a medical student answering an exam question about writing clinical notes for patients.
<</SYS>>

Keep in mind that your answer will be asssessed based on incorporating all the provided information and the quality of prose.

1. Use prose to write an example clinical note for this patient's doctor.
2. Use less than three sentences.
3. Do not provide a diagnosis or recommendations.
4. Use the following information:

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm

## Run inference

In [6]:
array = chain.batch(batch)
output = json.dumps(array)

Clinical Note:
Ms. Rachael Ankunding presents today with a 3-day history of right-sided otitis media. The patient reports difficulty hearing and mild discomfort in the affected ear. Medical history includes seasonal allergies and a previous episode of otitis media 6 months ago.Clinical Note:
Mr. Gino Sawayn presents with acute bronchitis, as evident by his cough and chest tightness. His date of birth is August 5, 2019, and he has been experiencing symptoms for the past 3 days. Further evaluation and treatment are necessary to manage his symptoms and prevent any potential complications.Certainly! Here is an example clinical note for Jetta Greenholt based on the provided information:

"Patient presents with a fracture of the left clavicle. The patient was involved in a fall from a height of approximately 3 meters. Patient reports severe pain and limited mobility in the affected arm. NHS number is 3717400924, date of birth is June 22, 2004. Given name is Jetta, family name is Greenholt."C

In [7]:
os.makedirs(os.path.dirname(path_output), exist_ok=True)

with open(path_output, "w") as f:
    f.write(output)